In [106]:
##三大法人
import requests
import pandas as pd
import re
def str_to_int(text):
    n = 0
    text = text.strip()
    t2 = re.sub(",","",text)
    if "-" in t2:
        n = -1*int(t2.strip('-'))
    else:
        n = int(t2)
    return n
def get_sanda_db(date='20200916'):
    #https://www.twse.com.tw/fund/T86?response=csv&date=20200916&selectType=01
    link = "https://www.twse.com.tw/fund/T86?response=csv&date="+date+"&selectType=01"
    #https://www.twse.com.tw/fund/TWT47U?response=csv&date=20200901&selectType=01
    r = requests.get(link)
    if r.status_code == 200:
        text = r.text
        lines = re.split("\r\n",text)
        lines = lines[1:-11]
        header = lines[0]
        data = lines[1:]
        lol = []
        for e in data:
            t = e.strip( '" ",="')
            t = t.strip( '"*",="')
            items = t.split('","')
            items[0] = items[0].strip()
            for c in range(2,len(items)) :
                items[c] = str_to_int(items[c])
            lol.append(items)
        headers = header.split('","')
        headers[0] = headers[0].strip('"')
        headers[-1] = headers[-1].strip('",')
        #print(headers,len(headers))
        #print(lol[-1],len(lol[-1]))
        df  =pd.DataFrame(lol,columns=headers)
        #df.index = df['證券代號']
    else:
        print("error",r.status_code)
        df = None
    return df

In [107]:

df = get_sanda_db()

In [108]:
df

,證券代號,證券名稱,外陸資買進股數(不含外資自營商),外陸資賣出股數(不含外資自營商),外陸資買賣超股數(不含外資自營商),外資自營商買進股數,外資自營商賣出股數,外資自營商買賣超股數,投信買進股數,投信賣出股數,投信買賣超股數,自營商買賣超股數,自營商買進股數(自行買賣),自營商賣出股數(自行買賣),自營商買賣超股數(自行買賣),自營商買進股數(避險),自營商賣出股數(避險),自營商買賣超股數(避險),三大法人買賣超股數
0,1101,台泥,7609353,5390879,2218474,0,0,0,466000,447000,19000,-1000391,326000,698391,-372391,12000,640000,-628000,1237083
1,1102,亞泥,2676000,984000,1692000,0,0,0,63000,194000,-131000,-429000,14000,344000,-330000,4000,103000,-99000,1132000
2,1108,幸福,268000,61000,207000,0,0,0,0,0,0,0,0,0,0,0,0,0,207000
3,1109,信大,109000,21000,88000,0,0,0,0,2000,-2000,-10000,0,10000,-10000,0,0,0,76000
4,1103,嘉泥,129000,79000,50000,0,0,0,0,0,0,0,0,0,0,0,0,0,50000
5,1110,東泥,54000,45000,9000,0,0,0,0,0,0,0,0,0,0,0,0,0,9000
6,1104,環泥,189000,204000,-15000,0,0,0,0,0,0,0,0,0,0,0,0,0,-15000


In [53]:
    date='20200916'
    link = "https://www.twse.com.tw/fund/TWT47U?response=csv&date="+date+"&selectType=01"
    #https://www.twse.com.tw/fund/TWT47U?response=csv&date=20200901&selectType=01
    r = requests.get(link)

In [ ]:
todo:
    1. build an increasing database
    2. interface for date/stock_id